## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim

In [267]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
#from MaaSSim.performance import kpi_pax_pricing
#from MaaSSim.performance import kpi_veh_pricing


import logging
import matplotlib.pyplot as plt


import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [268]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 180 # 1 hour of simulation
params.times.patience = 180
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 100 # reuqests (and passengers)
params.nV = 10 # vehicles


## Parameters for ExMAS

In [269]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2 

#params.platforms.comm_rate = 0.2 
#params.shareability.comm_rate = float(0.30)

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True


# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)

In [270]:
params.kpi = 3

In [271]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

07-08-23 10:30:40-INFO-Initializing pairwise trip shareability between 100 and 100 trips.
07-08-23 10:30:40-INFO-creating combinations
07-08-23 10:30:40-INFO-9900	 nR*(nR-1)
07-08-23 10:30:40-INFO-Reduction of feasible pairs by 99.42%
07-08-23 10:30:40-INFO-Degree 2 	Completed
07-08-23 10:30:40-INFO-trips to extend at degree 2 : 141
07-08-23 10:30:40-INFO-At degree 2 feasible extensions found out of 2 searched
07-08-23 10:30:40-INFO-Degree 3 	Completed
07-08-23 10:30:40-INFO-trips to extend at degree 3 : 2
07-08-23 10:30:40-INFO-At degree 3 feasible extensions found out of 0 searched
07-08-23 10:30:40-INFO-Degree 4 	Completed
07-08-23 10:30:40-INFO-Max degree reached 4
07-08-23 10:30:40-INFO-Trips still possible to extend at degree 4 : 0


In [272]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

07-08-23 10:30:40-INFO-Matching 100 trips to 243 rides in order to min u_veh
07-08-23 10:30:40-INFO-Problem solution: Optimal. 
Total costs for single trips:         27,048 
reduced by matching to:               23,754


In [273]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],2.932000,158,1,[2.932],"[0, 158]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1576709875, 503608204]","[None, 0, 0]",node time req_id od 0 ...,158,1.58,2.379000,0.475800,1.9032
1,[1],4.776000,258,1,[4.776],"[54, 258]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1620840079, 1402771629]","[None, 1, 1]",node time req_id od 0 ...,258,2.58,3.873000,0.774600,3.0984
2,[2],6.678500,361,1,[6.6785],"[67, 361]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44903734, 1413910852]","[None, 2, 2]",node time req_id od 0 ...,361,3.61,5.415000,1.083000,4.3320
3,[3],5.531500,299,1,[5.5315],"[95, 299]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44854744, 44813102]","[None, 3, 3]",node time req_id od 0 NaN...,299,2.99,4.485000,0.897000,3.5880
4,[4],3.908000,211,1,[3.9080000000000004],"[132, 211]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1668136752, 44786849]","[None, 4, 4]",node time req_id od 0 ...,211,2.11,3.169500,0.633900,2.5356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,"[58, 71]",11.230175,791,21,"[9.88135, 1.3488250000000002]","[2335.0, 330, 83, 378]","[58, 71]","[71, 58]",2,238,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44810363, 1393024904, 1474883280, 59138...","[None, 58, 71, 71, 58]",node time req_id od 0 ...,791,7.91,7.198875,1.439775,5.7591
239,"[58, 69]",12.553600,807,21,"[10.179375, 2.374225]","[2285.5, 366, 134, 307]","[58, 69]","[69, 58]",2,239,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44810363, 1569975455, 4349049083, 59138...","[None, 58, 69, 69, 58]",node time req_id od 0 ...,807,8.07,7.767000,1.553400,6.2136
240,"[58, 64]",14.535175,863,21,"[10.2226, 4.312575]","[2350.0, 193, 270, 400]","[58, 64]","[64, 58]",2,240,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44810363, 1413910820, 5235337217, 59138...","[None, 58, 64, 64, 58]",node time req_id od 0 ...,863,8.63,9.298125,1.859625,7.4385
241,"[13, 7, 21]",21.759950,753,32,"[6.831125, 9.128325, 5.8004999999999995]","[179.5, 140, 240, 44, 280, 49]","[13, 7, 21]","[13, 21, 7]",3,241,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",1,"[None, 44786382, 44817967, 1371031431, 4846026...","[None, 13, 7, 21, 13, 21, 7]",node time req_id od 0 ...,753,7.53,14.063625,2.812725,11.2509


# calculations 

In [274]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],2.932000,158,1,[2.932],"[0, 158]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1576709875, 503608204]","[None, 0, 0]",node time req_id od 0 ...,158,1.58,2.379000,0.475800,1.9032
1,[1],4.776000,258,1,[4.776],"[54, 258]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1620840079, 1402771629]","[None, 1, 1]",node time req_id od 0 ...,258,2.58,3.873000,0.774600,3.0984
2,[2],6.678500,361,1,[6.6785],"[67, 361]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44903734, 1413910852]","[None, 2, 2]",node time req_id od 0 ...,361,3.61,5.415000,1.083000,4.3320
3,[3],5.531500,299,1,[5.5315],"[95, 299]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44854744, 44813102]","[None, 3, 3]",node time req_id od 0 NaN...,299,2.99,4.485000,0.897000,3.5880
4,[4],3.908000,211,1,[3.9080000000000004],"[132, 211]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1668136752, 44786849]","[None, 4, 4]",node time req_id od 0 ...,211,2.11,3.169500,0.633900,2.5356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,"[58, 71]",11.230175,791,21,"[9.88135, 1.3488250000000002]","[2335.0, 330, 83, 378]","[58, 71]","[71, 58]",2,238,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44810363, 1393024904, 1474883280, 59138...","[None, 58, 71, 71, 58]",node time req_id od 0 ...,791,7.91,7.198875,1.439775,5.7591
239,"[58, 69]",12.553600,807,21,"[10.179375, 2.374225]","[2285.5, 366, 134, 307]","[58, 69]","[69, 58]",2,239,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44810363, 1569975455, 4349049083, 59138...","[None, 58, 69, 69, 58]",node time req_id od 0 ...,807,8.07,7.767000,1.553400,6.2136
240,"[58, 64]",14.535175,863,21,"[10.2226, 4.312575]","[2350.0, 193, 270, 400]","[58, 64]","[64, 58]",2,240,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44810363, 1413910820, 5235337217, 59138...","[None, 58, 64, 64, 58]",node time req_id od 0 ...,863,8.63,9.298125,1.859625,7.4385
241,"[13, 7, 21]",21.759950,753,32,"[6.831125, 9.128325, 5.8004999999999995]","[179.5, 140, 240, 44, 280, 49]","[13, 7, 21]","[13, 21, 7]",3,241,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",1,"[None, 44786382, 44817967, 1371031431, 4846026...","[None, 13, 7, 21, 13, 21, 7]",node time req_id od 0 ...,753,7.53,14.063625,2.812725,11.2509


In [275]:
sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate

07-08-23 10:30:43-INFO-Matching 100 trips to 243 rides in order to min u_veh
07-08-23 10:30:43-INFO-Problem solution: Optimal. 
Total costs for single trips:         27,048 
reduced by matching to:               23,754
0.0005
07-08-23 10:30:46-CRITICAL-this is request 0 with [0, 110] available rides.
07-08-23 10:30:46-CRITICAL-ride 0 available [1]
07-08-23 10:30:46-CRITICAL-ride 110 available [1, 0]
07-08-23 10:30:46-CRITICAL-this is request 0 with [0, 110] still available rides.
07-08-23 10:30:46-CRITICAL-vehicle 1 has 2 choices
0.0005
07-08-23 10:30:46-CRITICAL-this is request 1 with [1, 151, 164] available rides.
07-08-23 10:30:46-CRITICAL-ride 1 available [1]
07-08-23 10:30:46-CRITICAL-ride 151 available [1, 0]
07-08-23 10:30:46-CRITICAL-ride 164 available [1, 0]
07-08-23 10:30:46-CRITICAL-this is request 1 with [1, 151, 164] still available rides.
07-08-23 10:30:46-CRITICAL-vehicle 8 has 3 choices
0.0005
07-08-23 10:30:46-CRITICAL-this is request 2 with [2, 163] available rides.
0

In [276]:
sim.res[0].pax_exp

,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,LOSES_PATIENCE,MEETS_DRIVER_AT_PICKUP,RECEIVES_OFFER,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,TRAVEL,WAIT,OPERATIONS,LOST_PATIENCE
pax,,,,,,,,,,,,,,,,,,
0,15.0,158.0,20.0,30.0,0.0,211.0,0.0,0.0,10.0,0,0,0,0,0,158.0,211.0,55.0,False
1,15.0,258.0,20.0,30.0,0.0,2.0,0.0,54.0,10.0,0,0,0,0,0,258.0,2.0,55.0,False
2,15.0,415.0,20.0,30.0,0.0,190.0,122.0,67.0,10.0,0,0,0,0,0,415.0,312.0,55.0,False
3,15.0,299.0,20.0,30.0,0.0,154.0,0.0,95.0,10.0,0,0,0,0,0,299.0,154.0,55.0,False
4,15.0,211.0,20.0,30.0,0.0,54.0,0.0,132.0,10.0,0,0,0,0,0,211.0,54.0,55.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,153.0,20.0,30.0,0.0,124.0,124.0,3343.0,10.0,0,0,0,0,0,153.0,248.0,40.0,False
96,0.0,181.0,13.0,30.0,0.0,0.0,3200.0,267.0,10.0,0,0,0,0,0,181.0,3200.0,40.0,False
97,0.0,379.0,35.0,30.0,0.0,0.0,3082.0,385.0,10.0,0,0,0,0,0,379.0,3082.0,40.0,False


In [277]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,DRIVING_TIME,DRIVING_DIST,FARE,COMMISSION,REVENUE,PICKUP_DIST,TRAVEL_DIST,OPERATING_COST,PROFIT,LOST_PATIENCE
veh,,,,,,,,,,,,,,,,,,,,,
1,2,0.0,False,0,0,20.0,15.0,0,60.0,0,...,1025.0,10.25,3.8730,0.7746,3.0984,7.94,10.52,0.5260,2.5724,False
2,5,0.0,False,0,0,329.0,53.0,0,149.0,0,...,2144.0,21.44,5.4150,1.0830,4.3320,12.16,15.77,0.7885,3.5435,False
3,9,0.0,False,0,0,328.0,30.0,0,278.0,0,...,3787.0,37.87,4.4850,0.8970,3.5880,19.16,22.15,1.1075,2.4805,False
4,3,0.0,False,0,0,152.0,15.0,0,110.0,0,...,854.0,8.54,3.1695,0.6339,2.5356,3.86,5.97,0.2985,2.2371,False
5,11,0.0,False,0,0,246.0,30.0,0,322.0,0,...,2810.0,28.10,2.8350,0.5670,2.2680,11.98,13.87,0.6935,1.5745,False
6,5,0.0,False,0,0,230.0,15.0,0,202.0,0,...,2517.0,25.17,2.7885,0.5577,2.2308,12.21,14.06,0.7030,1.5278,False
7,8,0.0,False,0,0,222.0,32.0,0,408.0,0,...,2571.0,25.71,7.5210,1.5042,6.0168,10.93,15.94,0.7970,5.2198,False
8,8,0.0,False,0,0,158.0,35.0,0,349.0,0,...,3129.0,31.29,6.3210,1.2642,5.0568,12.48,16.69,0.8345,4.2223,False
9,8,0.0,False,0,0,445.0,45.0,0,294.0,0,...,2770.0,27.70,5.9250,1.1850,4.7400,11.30,15.25,0.7625,3.9775,False


# Service rate 

In [278]:
Fulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[False]
unFulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')



service_rate =  67.0 %


In [279]:
for i in range(1, 2):
    params.kpi = i
    #inData = ExMAS.main(inData, params.shareability, plot = False)
    #inData = prep_shared_rides(inData, params.shareability) # prepare schedules
   # inData.sblts.rides
   # sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate
    #params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
   # sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))

In [280]:
#sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv')
#sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv')

In [281]:
#for i in range(1, 2):
    #params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
   # sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    #sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
   # sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))
   

In [282]:
#sim.vehs[2].rdf 

In [283]:
#pd.DataFrame(sim.vehs[2].myrides)

## Script 